In [1]:
from torchvision.io import read_image
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Subset
import torch
import pickle
from generator import generator, gen_with_attn
from dataset import PixelSceneryDataset
from torchsummary import summary
%load_ext autoreload
%autoreload 2

In [ ]:
Train_Dir_live="imagenet/"
Train_Dir_pix="pixel/"

#setting up dataset
dataset = PixelSceneryDataset(
        root_scenery=Train_Dir_live,
        root_pixel=Train_Dir_pix,
        
)

with open('train_subset.pkl', 'rb') as f:
        train_indices = pickle.load(f)

with open('val_subset.pkl', 'rb') as f:
        val_indices = pickle.load(f)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

In [2]:
G_live = generator()
summary(G_live, (3, 256, 256), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           3,136
            Conv2d-2          [-1, 128, 64, 64]         131,200
       BatchNorm2d-3          [-1, 128, 64, 64]             256
            Conv2d-4          [-1, 256, 32, 32]         524,544
       BatchNorm2d-5          [-1, 256, 32, 32]             512
            Conv2d-6          [-1, 512, 16, 16]       2,097,664
       BatchNorm2d-7          [-1, 512, 16, 16]           1,024
            Conv2d-8            [-1, 512, 8, 8]       4,194,816
       BatchNorm2d-9            [-1, 512, 8, 8]           1,024
           Conv2d-10            [-1, 512, 4, 4]       4,194,816
      BatchNorm2d-11            [-1, 512, 4, 4]           1,024
           Conv2d-12            [-1, 512, 2, 2]       4,194,816
      BatchNorm2d-13            [-1, 512, 2, 2]           1,024
           Conv2d-14            [-1, 51

In [17]:
G_new = gen_with_attn()
summary(G_new, (3, 256, 256), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           3,136
       BatchNorm2d-2         [-1, 64, 128, 128]             128
         LeakyReLU-3         [-1, 64, 128, 128]               0
            Conv2d-4          [-1, 128, 64, 64]         131,200
       BatchNorm2d-5          [-1, 128, 64, 64]             256
         LeakyReLU-6          [-1, 128, 64, 64]               0
          ConvDown-7          [-1, 128, 64, 64]               0
            Conv2d-8          [-1, 128, 64, 64]         147,584
       BatchNorm2d-9          [-1, 128, 64, 64]             256
             ReLU-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]         147,584
      BatchNorm2d-12          [-1, 128, 64, 64]             256
             ReLU-13          [-1, 128, 64, 64]               0
    residualBlock-14          [-1, 128,

In [ ]:
checkpoint = torch.load('train_result.pth')
G_live = generator()
G_live.load_state_dict(checkpoint['G_live_state_dict'])
G_pix = generator()
G_pix.load_state_dict(checkpoint['G_pix_state_dict'])


In [ ]:
pixel, scenery = val_dataset[135]

In [ ]:
plt.imshow(np.moveaxis(scenery.numpy(), 0, -1))

In [ ]:
save_image(G_pix(scenery[None,...]) * 0.5 + 0.5, 'live_gen.png')